In [37]:
# %pip install tensorflow
# %pip freeze > requirements.txt

In [38]:
# generate training set
from tensorflow import keras

train_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/2000-samples-bined/',
	validation_split=0.2,
	subset="training",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)
train_set.class_names


Found 2000 files belonging to 2 classes.
Using 1600 files for training.


['no whale', 'whale present']

In [39]:
# generate validation set
validation_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/2000-samples-bined/',
	validation_split=0.2,
	subset="validation",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)

Found 2000 files belonging to 2 classes.
Using 400 files for validation.


In [40]:
for images, labels in train_set:
    preprocessed_images = keras.applications.resnet50.preprocess_input(images)

preprocessed_images, labels


(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
 array([[[[  -4.939003 ,  -34.779    ,  107.32     ],
          [  -4.939003 ,  -34.779    ,  107.32     ],
          [  -8.939003 ,  -26.779    ,  113.32     ],
          ...,
          [  -9.939003 ,    8.221001 ,  126.32     ],
          [   8.060997 ,  -52.779    ,   83.32     ],
          [   8.060997 ,  -52.779    ,   83.32     ]],
 
         [[  -4.939003 ,  -34.779    ,  107.32     ],
          [  -4.939003 ,  -34.779    ,  107.32     ],
          [  -8.939003 ,  -26.779    ,  113.32     ],
          ...,
          [  -9.939003 ,    8.221001 ,  126.32     ],
          [   8.060997 ,  -52.779    ,   83.32     ],
          [   8.060997 ,  -52.779    ,   83.32     ]],
 
         [[  -4.939003 ,  -34.779    ,  107.32     ],
          [  -4.939003 ,  -34.779    ,  107.32     ],
          [  -8.939003 ,  -26.779    ,  113.32     ],
          ...,
          [  -9.939003 ,    8.221001 ,  126.32     ],
          [   8.060997 , 

In [41]:
# load resnet50 model with pretrained weights
# replace output layer with 2 neurons
# connect to gpu?
# shuffle training set
#  cross entrpoy loss, otimize adam model parameters learning rate 0.0001 epochs 10
# save model
# confusion matrix

In [42]:
# base_model = keras.applications.ResNet50(
# 	weights= None, 
# 	# include_top=False, 
# 	# input_shape= (img_height,img_width,3)
# 	num_classes=2
# 	)

imported_model= keras.applications.ResNet50(include_top=False,
	input_shape=(224,224,3),
	pooling='avg',
	classes=2,
	weights='imagenet'
)

In [43]:
custom_model = keras.models.Sequential()
custom_model.add(imported_model)
custom_model.add(keras.layers.Dense(2, activation='softmax'))
custom_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 23538690 (89.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [44]:
from keras.optimizers.legacy import Adam
custom_model.compile(
	optimizer=Adam(learning_rate=0.001),
	loss='sparse_categorical_crossentropy',
	metrics=['accuracy']
)

In [73]:
history = custom_model.fit(
	train_set,
	validation_data=validation_set,
	epochs=10
)

Epoch 1/10
50/50 [==============================] - 133s 3s/step - loss: 0.0633 - accuracy: 0.9756 - val_loss: 0.4477 - val_accuracy: 0.9450
Epoch 2/10
50/50 [==============================] - 131s 3s/step - loss: 0.0477 - accuracy: 0.9831 - val_loss: 0.1488 - val_accuracy: 0.9625
Epoch 3/10
50/50 [==============================] - 131s 3s/step - loss: 0.0298 - accuracy: 0.9894 - val_loss: 0.1763 - val_accuracy: 0.9500
Epoch 4/10
50/50 [==============================] - 132s 3s/step - loss: 0.0309 - accuracy: 0.9881 - val_loss: 0.5193 - val_accuracy: 0.9125
Epoch 5/10
50/50 [==============================] - 132s 3s/step - loss: 0.0174 - accuracy: 0.9944 - val_loss: 0.6627 - val_accuracy: 0.8000
Epoch 6/10
50/50 [==============================] - 132s 3s/step - loss: 0.0706 - accuracy: 0.9756 - val_loss: 5.4009 - val_accuracy: 0.9225
Epoch 7/10
50/50 [==============================] - 128s 3s/step - loss: 0.0366 - accuracy: 0.9875 - val_loss: 0.3111 - val_accuracy: 0.9000
Epoch 8/10
50

In [74]:
# save custom_model
custom_model.save('resnet50_epochs10_samples2000_seed123.h5')

/Users/lshuster/Library/CloudStorage/OneDrive-GlobalHealthcareExchange/Desktop/whale-detection/.conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
from sklearn.metrics import confusion_matrix
import numpy as np

#Predict
y_prediction = custom_model.predict(validation_set)

# folder = '../../input/numpy/2000-samples/'
# y = np.load(folder+'y.npy')

#Create confusion matrix and normalizes it over predicted (columns)
# result = confusion_matrix(y_test, y_prediction , normalize='pred')


13/13 [==============================] - 8s 619ms/step


In [56]:
y_prediction
y_prediction.shape
y_pred = np.argmax(y_prediction, axis=1)
y_pred

array([0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [ ]:
# Generate Matrix

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_test, y_pred)
labels = ['No Whale','Whale Present']

sns.heatmap(
	cf_matrix, 
	annot=True,
	fmt='d',
	xticklabels=labels,
	yticklabels=labels,
	cmap='Blues'
).set(
	xlabel='Prediction',
	ylabel='Actual'
)
plt.show()

sns.heatmap(
	cf_matrix/np.sum(cf_matrix), 
	annot=True,
	fmt='.2%',
	xticklabels=labels,
	yticklabels=labels,
	cmap='Blues'
).set(
	xlabel='Prediction',
	ylabel='Actual'
)
plt.show()
